In [2]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Unclassified Splits/Unattributable Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

In [3]:
excel_path_unattributable = path_equity_precision_llm_folder + "//llm training/epldat03_Unattributable Data.csv"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

In [4]:
input_unattributable = pd.read_csv(excel_path_unattributable)
input_unattributable.head()

,PMID,MeSH,Abstract,Title,year
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010


In [17]:
import pandas as pd
import re

n_json_splits = 11
# Concatenate multiple CSVs
results = None
for scenario in range(1, n_json_splits + 1):
    file_path = f"{path_equity_precision_llm_folder}/llm training/Unclassified Splits/Unattributable Part {scenario}_results.csv"
    temp_df = pd.read_csv(file_path)
    temp_df = temp_df[['pmid', 'title', 'gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study', 'gpt_source_population']]

    if results is None:
        results = temp_df
    else:
        results = pd.concat([results, temp_df], ignore_index=True)

# Merge datasets
merged_df_unattributable = input_unattributable.merge(results, left_on='PMID', right_on='pmid', how='left')
merged_df_unattributable['gpt_source_population'] = merged_df_unattributable['gpt_source_population'].apply(standardize_population)

In [18]:
unique_source_populations = merged_df_unattributable['gpt_source_population'].unique()
print(unique_source_populations)

['Unknown' 'we' 'ea' 'na' 'mena' 'not applicable (animal study)' 'cee'
 'ea, ssa, sa, we, na, cee, mena, seap, ca, lac'
 'not applicable (study conducted on rats, not humans)' '-' 'sa' 'none'
 'ssa' 'na, lac' 'not determinable' 'lac' 'seap' 'ea, sa' 'ea, we'
 'excluded' 'we, ssa' 'not applicable (non-human study)' 'sa, ssa, we'
 'ssa, we' 'na, we' 'not applicable (n/a)' 'lac, ssa' 'ns' 'sa, we'
 'not applicable (non-human animal model)'
 'not applicable (study based on animal model)' 'lac, mena, we' 'sa, na'
 'not applicable (na)' 'ssa, we, ea, lac' 'ea, na' 'not discernible'
 'ea, sa, we' 'not applicable (study on animal models)'
 'not applicable (study uses 3t3-l1 preadipocytes, which are a mouse cell line)'
 'mena, sa' 'we, na' 'na, we, seap' 'we, sa' 'u'
 'not applicable (study does not involve human participants)'
 'we, ea, seap, lac' 'ea, seap' 'ssa, na' 'cee, we' 'undetermined'
 'lac, mena' 'not applicable (study uses non-human animal models)' 'ca'
 'n/a (animal study)' 'not eno

In [19]:
# Check the output below
merged_df_unattributable.head()

,PMID,MeSH,Abstract,Title,year,pmid,title,gpt_precision_medicine,gpt_diabetes,gpt_primary_study,gpt_source_population
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013,18008027.0,Renoprotective effects of telmisartan on rena...,no,yes,yes,Unknown
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011,18205795.0,CA 72-4 levels in patients with type 2 diabet...,no,yes,yes,Unknown
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010,18378359.0,Vascular risk factors and dementia in the gen...,no,yes,yes,we
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010,18626579.0,Kinetic analysis and pH-shift control strateg...,no,no,yes,Unknown
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010,18651247.0,Production of butyric acid from glucose and x...,no,no,yes,Unknown


In [23]:
import pandas as pd

# Create a crosstab summary function
def crosstab_summary(merged_df_unattributable, categories, region_col='gpt_source_population'):
    """
    Generate a crosstab summary of studies categorized by multiple conditions, stratified by region.
    
    :param df: DataFrame containing the study data
    :param categories: List of column names representing different study categories
    :param region_col: Column name representing the region for stratification
    :return: DataFrame summarizing counts of different study combinations by region
    """
    merged_df_unattributable['combined_category'] = merged_df_unattributable[categories].astype(str).agg('_'.join, axis=1)
    summary = pd.crosstab(merged_df_unattributable['combined_category'], merged_df_unattributable[region_col])
    summary.reset_index(inplace=True)
    return summary

# Define categories
categories = ['gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study']

# Generate crosstab summary stratified by region
summary_merged_df_unattributable = crosstab_summary(merged_df_unattributable, categories, region_col='gpt_source_population')

# Save the output
output_path = path_equity_precision_llm_repo + '/preprocessing/epl05_combined_output_Unattributable.csv'
summary_merged_df_unattributable.to_csv(output_path, index=False)

# Display summary
display(summary_merged_df_unattributable)

gpt_source_population,combined_category,-,Unknown,[ca/cee/ea/lac/mena/seap/sa/ssa/na/we],[not available],ca,"ca, cee, ea, lac, mena, seap, sa, ssa","ca, cee, ea, lac, mena, seap, sa, ssa, na, we","ca, cee, ea, lac, mena, seap, sa, ssa, we, na","ca, cee, ea, lac, na, sa, ssa, we",...,"we, seap","we, ssa","we, ssa, ea, sa, na","we, ssa, lac, mena","we, ssa, na","we, ssa, na, sa, ea, mena","we, ssa, sa","we, ssa, sa, mena",we/na,we/na (assumed)
0,[yes/no]_[yes/no]_[yes/no],0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,nan_nan_nan,0,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,no (insufficient information)_no (insufficient...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,no evidence due to retraction_no evidence due ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,no information_no information_no information,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,no_no_no,231,6860,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6,no_no_not applicable,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,no_no_not assessable,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,no_no_not determinable from given information,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,no_no_not enough information,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
merged_df_unattributable.to_csv(path_equity_precision_llm_folder + '/llm training/epl05_output_Unattributable.csv', index=False)